# Part C
#### Grading Criteria
Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?

In [29]:
# perform installs
!pip install numpy==2.0.2
!pip install pandas==2.2.2
!pip install tensorflow_cpu==2.18.0
!pip install scikit-learn

In [30]:
# imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

import warnings
warnings.simplefilter('ignore', FutureWarning)

In [31]:
# Get concrete dataset
filepath='https://cocl.us/concrete_data'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [32]:
# check how many data points we have
concrete_data.shape

(1030, 9)

In [33]:
# look at dataset info
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [34]:
# make sure we don't have null values
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [35]:
# Split dataset into predictors (all columns except Strength) and target (Strength column)
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [36]:
# normalize predictors
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [37]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

In [38]:
# function to build model
def build_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    # model has hidden layer with 10 nodes and relu act func
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error') # Use adam and mse
    return model

In [39]:
# build model
model = build_model()

In [40]:
mse_list = []

In [41]:
def test_mse():
    # split into train and test sets and hold 30% of data for testing
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)
    # train model with 100 epochs now
    model.fit(X_train, y_train, epochs=100, verbose=2)
    y_pred = model.predict(X_test)  #eval model on test data
    mse = mean_squared_error(y_test, y_pred)  # compute mse
    mse_list.append(mse)  #add to list of mses

In [42]:
# repeat 50 times
num_iterations = 50
for i in range(num_iterations):
    test_mse()

Epoch 1/100
23/23 - 1s - 30ms/step - loss: 1600.3428
Epoch 2/100
23/23 - 0s - 5ms/step - loss: 1583.7219
Epoch 3/100
23/23 - 0s - 5ms/step - loss: 1566.9576
Epoch 4/100
23/23 - 0s - 5ms/step - loss: 1550.4377
Epoch 5/100
23/23 - 0s - 4ms/step - loss: 1533.8329
Epoch 6/100
23/23 - 0s - 4ms/step - loss: 1516.9006
Epoch 7/100
23/23 - 0s - 4ms/step - loss: 1499.9696
Epoch 8/100
23/23 - 0s - 4ms/step - loss: 1482.3110
Epoch 9/100
23/23 - 0s - 4ms/step - loss: 1464.4423
Epoch 10/100
23/23 - 0s - 4ms/step - loss: 1445.9709
Epoch 11/100
23/23 - 0s - 4ms/step - loss: 1426.7662
Epoch 12/100
23/23 - 0s - 4ms/step - loss: 1406.9587
Epoch 13/100
23/23 - 0s - 5ms/step - loss: 1386.4410
Epoch 14/100
23/23 - 0s - 4ms/step - loss: 1365.5875
Epoch 15/100
23/23 - 0s - 4ms/step - loss: 1343.5474
Epoch 16/100
23/23 - 0s - 4ms/step - loss: 1321.2325
Epoch 17/100
23/23 - 0s - 4ms/step - loss: 1298.2594
Epoch 18/100
23/23 - 0s - 4ms/step - loss: 1274.4330
Epoch 19/100
23/23 - 0s - 4ms/step - loss: 1250.2051
E

In [43]:
# Report the mean and the standard deviation of the mean squared errors.
average_mse = np.mean(mse_list)
print(f"Average Mean Squared Error over {num_iterations} iterations: {average_mse}")
std_mse = np.std(mse_list)
print(f"Mean Squared Error Standard Deviation over {num_iterations} iterations: {std_mse}")

Average Mean Squared Error over 50 iterations: 40.26975296753347
Mean Squared Error Standard Deviation over 50 iterations: 20.859128230630006


In [1]:
# Part B metrics:
# Average Mean Squared Error over 50 iterations: 51.930608871813966
# Mean Squared Error Standard Deviation over 50 iterations: 36.85541987092113

# How does the mean of the mean squared errors compare to that from Step B?
For part B, the mean of the mean squared errors was 51.930608871813966, and the standard deviation was 36.85541987092113.
For part C, the mean of the mean squared errors was 40.26975296753347, and the standard deviation was 20.859128230630006.
The lower mean and lower standard deviation for part C indicate that increasing the number of epochs from 50 to 100 improved the model's
performance.